In [49]:
!pip install requests
!pip install beautifulsoup4
!pip install selenium

In [50]:
pip install --upgrade selenium

Note: you may need to restart the kernel to use updated packages.


In [51]:
# packages
import requests
from bs4 import BeautifulSoup as BS
from selenium import webdriver
import pandas as pd
import numpy as np
import os
import warnings
warnings.filterwarnings('ignore')

In [52]:
# 스타벅스 url
url = 'https://www.starbucks.co.kr/store/store_map.do'

In [53]:
chorme_driver = './chromedriver_win32/chromedriver.exe'
driver = webdriver.Chrome(chorme_driver)

# url open
driver.get(url)

# 최대 7초 대기
driver.implicitly_wait(7)

In [54]:
import time

# '지역 검색' 탭 클릭하기
link_element = driver.find_element('link text', '지역 검색')
link_element.click()

# 서울 전체 클릭
search =driver.find_element('link text', '서울')
search.click()

# 5초 대기
time.sleep(5)

search =driver.find_element('link text', '전체')
search.click()

In [55]:
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.action_chains import ActionChains

# 웹 페이지 맨 아래로 스크롤 할 수 있도록 함수 정의
def scroll_down():
    actions = ActionChains(driver)
    actions.send_keys(Keys.END).perform()
    time.sleep(3)

for i in range(3):
    scroll_down()

In [56]:
# 파싱
html_content = driver.page_source
content = BS(html_content, 'html.parser')

# 닫기
driver.quit()

# 데이터 수집

In [57]:
# 서울 스타벅스 개수
starbucks_cnt = int(content.select('.sidoSetResult')[0].get_text(strip=True))
starbucks_cnt

611

In [58]:
import re
starbucks = []
stores = content.select('li[data-name]')
for store in stores:
    name = store['data-name']
    y = store['data-lat'] # 위도
    x = store['data-long'] #경도
    
    string = store.select('.result_details')[0].get_text(strip=True)
    split_result = re.split(r'(\d{4}-\d{4})', string)
    address = split_result[0].strip()
    number = split_result[1].strip()
    starbucks.append({'name': name, 
                    'address': address,
                    'number': number,
                    'x':x,
                    'y':y})
    
starbucks = pd.DataFrame(starbucks)

In [59]:
# 결측치 확인하기
starbucks.isna().sum()

name       0
address    0
number     0
x          0
y          0
dtype: int64

In [60]:
# 중복 삭제
starbucks = starbucks[~starbucks.duplicated(keep='last')]

In [61]:
# 경기도 제외
starbucks = starbucks[~starbucks['address'].str.contains('경기도')]

In [62]:
# 필요없는 number 컬럼 삭제
col = [col for col in starbucks.columns if col !='number']
starbucks = starbucks[col]

In [63]:
starbucks

,name,address,x,y
10,역삼아레나빌딩,서울특별시 강남구 언주로 425 (역삼동),127.043069,37.501087
11,논현역사거리,서울특별시 강남구 강남대로 538 (논현동),127.022223,37.510178
12,신사역성일빌딩,서울특별시 강남구 강남대로 584 (논현동),127.0206057,37.5139309
13,국기원사거리,서울특별시 강남구 테헤란로 125 (역삼동),127.031495,37.499517
14,대치재경빌딩,서울특별시 강남구 남부순환로 2947 (대치동),127.062583,37.494668
...,...,...,...,...
616,사가정역,서울특별시 중랑구 면목로 310,127.087966,37.579594
617,상봉역,서울특별시 중랑구 망우로 307 (상봉동),127.08647,37.59689
618,묵동,"서울특별시 중랑구 동일로 952 (묵동, 로프트원 태릉입구역) 1층",127.076633,37.615368
619,양원역,서울특별시 중랑구 양원역로10길 3 (망우동),127.106359790053,37.6066536267232


In [32]:
# 데이터 저장
starbucks.to_csv('./starbucks_data.csv', encoding='cp949', index=False)